In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
path = './data/'
p1= pd.read_csv(path+'users.csv')
p2= pd.read_csv(path+'books.csv')
p3= pd.read_csv(path+'train_ratings.csv')
p4= pd.read_csv(path+'sample_submission.csv')
p5= pd.read_csv(path+'test_ratings.csv') # test와 sample은 완벽하게 동일하다.

#조금 전처리됨
p6= pd.read_csv('/opt/ml/input/code/pre_users.csv')
p7= pd.read_csv('/opt/ml/input/code/books_fillna_cate.csv') 
p8= pd.read_csv('/opt/ml/input/code/books_lang.csv') 

users  =p6.copy()
books  =p7.copy()
train  =p3.copy()
sample =p4.copy()
test   =p5.copy()

boks =p8.copy()

print('users shape: ', users.shape) 
print(users.columns)#유저id, 지역, 나이
print('books shape: ', books.shape)
print(books.columns)#isbn(책번호), 제목, 저자, 출판년도, 출판사, img주소, 언어, 범주, 요약, img경로
print('train shape: ', train.shape) 
print(train.columns)# 유저 id, 책isbn, 평점

users shape:  (68092, 5)
Index(['user_id', 'age', 'city', 'state', 'country'], dtype='object')
books shape:  (149570, 9)
Index(['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher',
       'language', 'category', 'summary', 'high_category'],
      dtype='object')
train shape:  (306795, 3)
Index(['user_id', 'isbn', 'rating'], dtype='object')


In [9]:
boks

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,...,vowel_density,capitals,caps_vs_length,num_exclamation_marks,num_question_marks,num_punctuation,num_unique_words,num_repeated_words,words_vs_unique,encode_ascii
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,...,1.000000,2,0.181818,0,0,0,2,0,1.000000,1.0
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg,...,1.000000,2,0.111111,0,0,0,3,0,1.000000,1.0
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg,...,0.894737,11,0.137500,0,0,0,16,2,0.842105,1.0
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg,...,1.000000,4,0.210526,0,0,1,4,0,1.000000,1.0
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg,...,0.916667,12,0.176471,0,1,2,11,1,0.916667,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,['Humor'],A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg,...,0.923077,11,0.177419,0,0,0,13,0,1.000000,1.0
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,['Nature'],A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg,...,0.800000,7,0.140000,0,0,0,10,0,1.000000,1.0
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,en,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg,...,0.777778,3,0.063830,0,1,1,9,0,1.000000,1.0
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,['Fiction'],These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg,...,0.909091,10,0.178571,0,1,3,11,0,1.000000,1.0


In [10]:
books['language'] = boks['language']

In [13]:
books['language']

en       143548
de         2541
es         1527
fr         1461
it          174
nl           98
pt           62
da           59
ca           42
ms           12
no            7
gl            7
ja            6
la            6
zh-CN         5
ru            4
ro            2
el            1
th            1
fa            1
eo            1
gd            1
ga            1
vi            1
zh-TW         1
ko            1
Name: language, dtype: int64

In [15]:
publisher_dict=(books['publisher'].value_counts()).to_dict()
publisher_count_df= pd.DataFrame(list(publisher_dict.items()),columns = ['publisher','count'])

publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False)
modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values
for publisher in (modify_list):
	try:
		number = books[books['publisher']==publisher]['isbn'].apply(lambda x: x[:4]).value_counts().index[0]
		right_publisher = books[books['isbn'].apply(lambda x: x[:4])==number]['publisher'].value_counts().index[0]
		books.loc[books[books['isbn'].apply(lambda x: x[:4])==number].index,'publisher'] = right_publisher
	except:
		pass

In [17]:
books = books.drop(['category'], axis=1)

In [20]:
books.columns = ['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher',
       'language', 'summary', 'category_high']

In [24]:
if not os.path.isfile('/opt/ml/input/code/books_fill.csv'):
	books.to_csv('/opt/ml/input/code/books_fill.csv', index=False)

In [23]:
train

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9
...,...,...,...
306790,278843,0743525493,7
306791,278851,067161746X,6
306792,278851,0884159221,7
306793,278851,0912333022,7


In [41]:
(users[users.isna()])

,user_id,age,city,state,country
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
68087,NaN,NaN,NaN,NaN,NaN
68088,NaN,NaN,NaN,NaN,NaN
68089,NaN,NaN,NaN,NaN,NaN
68090,NaN,NaN,NaN,NaN,NaN


In [34]:
(users[users['country'].isna() & users['city'].notna()] )

,user_id,age,city,state,country
1879,181316,36.0,managua,NaN,NaN
2020,205019,29.0,aladinma,imostate,NaN
2183,227295,NaN,llanelli,NaN,NaN
2683,273110,51.0,arco,NaN,NaN
2743,53456,NaN,neportland,oregon,NaN
...,...,...,...,...,...
64760,133843,16.0,letart,westvirginia,NaN
65073,143210,16.0,orleans,NaN,NaN
65142,146516,31.0,saronno,lombardia,NaN
66518,192259,36.0,grossenseebach,NaN,NaN
